In [2]:
import os
import pandas as pd

from constants import RAW_IMAGES_DIRPATH, IMAGE_DATA_FILEPATH

In [3]:
def get_image_data(image_dirpath):
    image_data = []

    i = 0
    for filename in os.listdir(image_dirpath):
        print(i, filename)
        filepath = os.path.join(image_dirpath, filename)
        timestamp = os.path.getctime(filepath)
        image_data.append([filename, timestamp])
        i += 1
    df = pd.DataFrame(image_data, columns=["name", "timestamp"])

    return df

images_df = get_image_data(RAW_IMAGES_DIRPATH)

0 2023-06-16_19-21-15.jpg
1 2023-06-16_19-21-16.jpg
2 2023-06-16_19-21-18.jpg
3 2023-06-16_19-21-19.jpg
4 2023-06-16_19-21-21.jpg
5 2023-06-16_19-21-22.jpg
6 2023-06-16_19-21-24.jpg
7 2023-06-16_19-21-25.jpg
8 2023-06-16_19-21-27.jpg
9 2023-06-16_19-21-28.jpg
10 2023-06-16_19-21-30.jpg
11 2023-06-16_19-21-31.jpg
12 2023-06-16_19-21-33.jpg
13 2023-06-16_19-21-34.jpg
14 2023-06-16_19-21-36.jpg
15 2023-06-16_19-21-37.jpg
16 2023-06-16_19-21-39.jpg
17 2023-06-16_19-21-40.jpg
18 2023-06-16_19-21-42.jpg
19 2023-06-16_19-21-43.jpg
20 2023-06-16_19-21-47.jpg
21 2023-06-16_19-21-49.jpg
22 2023-06-16_19-21-50.jpg
23 2023-06-16_19-21-52.jpg
24 2023-06-16_19-21-53.jpg
25 2023-06-16_19-21-55.jpg
26 2023-06-16_19-21-56.jpg
27 2023-06-16_19-21-58.jpg
28 2023-06-16_19-21-59.jpg
29 2023-06-16_19-22-01.jpg
30 2023-06-16_19-22-02.jpg
31 2023-06-16_19-22-04.jpg
32 2023-06-16_19-22-05.jpg
33 2023-06-16_19-22-07.jpg
34 2023-06-16_19-22-08.jpg
35 2023-06-16_19-22-10.jpg
36 2023-06-16_19-22-11.jpg
37 2023-06-

In [4]:
images_df.sort_values(by="timestamp", inplace=True)

In [5]:
images_df["time_since_last"] = images_df["timestamp"].diff()

In [6]:
images_df["long_wait"] = images_df["time_since_last"] > 60

In [7]:
images_df["capture"] = images_df["long_wait"].cumsum()

In [9]:
images_df["first_in_capture"] = images_df.groupby("capture")["name"].transform("first")

In [10]:
images_df["first_in_capture"].value_counts().sort_values()

2023-06-22_07-14-22.jpg     21
2023-07-10_19-16-54.jpg     73
2023-07-01_13-14-03.jpg     80
2023-06-17_13-14-24.jpg    100
2023-06-17_19-14-24.jpg    100
                          ... 
2023-07-20_01-14-09.jpg    100
2023-07-20_07-14-07.jpg    100
2023-07-12_16-02-32.jpg    100
2023-07-12_07-16-10.jpg    100
2023-07-03_19-15-25.jpg    100
Name: first_in_capture, Length: 132, dtype: int64

In [11]:
images_df["capture_size"] = images_df.groupby("first_in_capture")["first_in_capture"].transform("count")

In [12]:
def get_capture_name(first: str, capture: int):
    date, time = first.split(".")[0].split("_")[:2]
    time = time.split("-")[0]
    return f"{capture}_{date}_{time}"

images_df["capture_name"] = images_df.apply(lambda row: get_capture_name(row["first_in_capture"], row["capture"]), axis=1)

In [14]:
images_df.to_csv(IMAGE_DATA_FILEPATH, index=False)

In [15]:
images_df["capture_name"].value_counts()

0_2023-06-16_19      100
99_2023-07-12_01     100
97_2023-07-11_13     100
96_2023-07-11_07     100
95_2023-07-11_01     100
                    ... 
36_2023-06-26_13     100
131_2023-07-20_13    100
56_2023-07-01_13      80
94_2023-07-10_19      73
22_2023-06-22_07      21
Name: capture_name, Length: 132, dtype: int64